# Evidencia 2

Nombres: Diego Lira García
        Andrea Medina Rico

## Importaciones

In [ ]:
%pip install mesa==2.3.1 --quiet
%pip install matplotlib numpy pandas --quiet
%pip install seaborn --quiet

In [3]:
# Importamos las clases que se requieren para manejar los agentes (Agent) y su entorno (Model).
# Cada modelo puede contener múltiples agentes.
from mesa import Agent, Model

# Debido a que necesitamos que existe un solo agente por celda, elegimos ''SingleGrid''.
from mesa.space import SingleGrid

# Con ''SimultaneousActivation, hacemos que todos los agentes se activen ''al mismo tiempo''.
from mesa.time import RandomActivation

# Haremos uso de ''DataCollector'' para obtener información de cada paso de la simulación.
from mesa.datacollection import DataCollector

# Librería para modificar parámetros --> Corre varias simulaciones
from mesa.batchrunner import batch_run

# matplotlib lo usaremos crear una animación de cada uno de los pasos del modelo.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime

## 1. Solución aleatoria

In [ ]:
class FirefighterAgent(Agent):
    def __init__(self, id, model):
        super().__init__(id, model)

        self.action_points = 4
        self.carrying_victim = False
        self.knocked_down = False

        # self.positions_not_available = [] ????

    
    def step(self):
        while self.action_points > 0:
            # Asignar un número a cada función y que elija aleatoriamente
            print("Teamo")


    def move_to_cell(self):
        possible_positions = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)
        indexes = [x for x in range( len(possible_positions) )]
        np.random.shuffle(indexes)

        for i in indexes:
            position = possible_positions[i]
            status = self._get_cell_status(position)
            walls_doors = self._check_walls_doors(position, i)

            if not walls_doors:
                if status == 2 and not self.carrying_victim:
                    self.model.grid.move_agent(self, position)
                    self.action_points -= 2
                elif self.carrying_victim and status != 2:
                    self.action_points -= 2
                    self.model.grid.move_agent(self, position)
                else:
                    self.action_points -= 1
                    self.model.grid.move_agent(self, position)
                break



    
    def toggle_door(self,position):
        """ Cambia el estado de la puerta en la celda (x, y). """
        x, y = position
        cell = self.model.doors[x][y]

        # Obtener cuatro posible celdas  --> 0 Arriba, 1 derecha, 2 abajo, 3 izquierda
        possible_positions = self.model.grid.get_neighborhood(self.pos, moore = False, include_center = False)

        # Encontrar la puerta que se quiere alterar
        # 0 Arriba, 1 izquierda, 2 abajo, 3 derecha
        # 1 Abierto 2 Cerrado
        index = 0
        for i in cell:
            if i != 0:
                if i == 1:
                    cell[index] = 2
                    new_state = 2
                else:
                    cell[index] = 1
                    new_state = 1

                self.action_points -= 1
                break
            index += 1
        
        # Encontrar la otra celda de la puerta
        if index == 0:
            other_cell = possible_positions[0]
            other_index = 2
        elif index == 2:
            other_cell = possible_positions[2]
            other_index = 0
        elif index == 1:
            other_cell = possible_positions[3]
            other_index = 3
        elif index == 3:
            other_cell = possible_positions[1]
            other_index = 1
        else:
            return
        
        # Cambiar el estado de la otra celda
        next_cell = self.model.doors[other_cell[0]][other_cell[1]]
        next_cell[other_index] = new_state
        return
    
    
    def extinguish(self, position, option):
        x, y = position
        cell = self.model.grid_status[x][y]
        
        if option == 1 and cell == 2 and self.action_points >= 2:
            self.model.grid_status[x][y] = 0
            self.action_points -= 2
        elif option == 2 and cell == 2 and self.action_points >= 1:
            self.model.grid_status[x][y] = 1
            self.action_points -= 1
        elif option == 3 and cell == 1 and self.action_points >= 1:
            self.model.grid_status[x][y] = 0
            self.action_points -= 1
        else:
            return
            

            
        


    #Metodos auxiliares

    def _check_walls_doors(self, position, index):
        x, y = position
        
        cell_doors = self.model.doors[x][y]
        cell_walls = self.model.walls[x][y]
        
        if index == 0:
            door = cell_doors[2]
            wall = cell_walls[2]
        elif index == 1:
            door = cell_doors[1]
            wall = cell_walls[1]
        elif index == 2:
            door = cell_doors[0]
            wall = cell_walls[0]
        elif index == 3:
            door = cell_doors[3]
            wall = cell_walls[3]
            
        if door == 2 or wall == 1 or wall == 0.5:
            return True
        else:
            return False
        
    def _get_cell_status(self, position):
        x, y = position
        return self.model.grid_status[x][y]
            


    